In [1]:
####################################################################################
######################## SIRA SCRIPT ###############################################
####################################################################################
import discord
from discord.ext import commands

import datetime
import os
import logging
import uuid
import asyncio
from dotenv import load_dotenv
import pytz
import pandas as pd
import numpy as np

# Google and Sheets-related imports
import gspread
from gcsa.google_calendar import GoogleCalendar
from gspread_formatting import (
    get_conditional_format_rules, 
    ConditionalFormatRule, 
    BooleanRule, 
    BooleanCondition, 
    CellFormat, 
    color
)

program = "SIFP_TEST"
gc = gspread.service_account(filename='service_account.json')
wks_PROG = gc.open(os.getenv(program.upper() + "_PROG_TOKEN"))
wks_SOG = gc.open(os.getenv(program.upper() + "_SOG_TOKEN"))


In [2]:
from EVENTS_EDIT_FUNCTIONS import conversion_excel_date, parse_times, get_color, post_events, get_event_by_search_query, update_events_by_id, Reorganize_Sheet, Verbose_Sheet, update_events_submitted, get_event_submitted
from EVENTS_IMPORT_FUNCTIONS import Import_Sheet, Reorganize_Sheet_Import, format_time_columns_as_time, Row_Offset


In [3]:
num_sheets = len(wks_SOG.worksheets())
n_template_sheets = 2
# 0. Copy Sheet 1 (TEMPLATE) and move it to final slot.
# if (num_sheets <)
print(num_sheets)

18


In [ ]:
from datetime import datetime as dt
from datetime import timedelta
week_number = 15
n_template_sheets = 2
num_sheets = len(wks_SOG.worksheets())

# Helper to convert python datetime to Excel/Google serial number
def to_excel_date(date_obj):
    epoch = dt(1899, 12, 30)
    delta = date_obj - epoch
    return delta.days + (delta.seconds / 86400.0)

existing_sheet = True
if (num_sheets < (week_number + n_template_sheets + 1)): 
    template_sheet = wks_SOG.worksheet("Template")
    existing_sheet = False
    start_date = dt.fromisoformat(os.getenv(program + "_START_DATE"))

    # a. Normalize to the very first Monday of the program
    if start_date.weekday() == 0:
        monday_date_0 = start_date
    else:
        days_to_subtract = start_date.weekday()
        monday_date_0 = start_date - timedelta(days=days_to_subtract)

    # b. Calculate this specific week's Monday and Sunday
    # (Moved outside the if/else so these always exist)
    monday_date = monday_date_0 + timedelta(days = 7 * (week_number - 1))
    sunday_date = monday_date + timedelta(days = 6)

    # c. Create the title string logic
    m_month = monday_date.strftime("%B")
    m_day = monday_date.strftime("%d").lstrip("0")
    s_month = sunday_date.strftime("%B")
    s_day = sunday_date.strftime("%d").lstrip("0")

    if monday_date.month == sunday_date.month:
        date_range = f"{m_month} {m_day}-{s_day}"
    else:
        date_range = f"{m_month} {m_day}-{s_month} {s_day}"

    title_string = f"Week {week_number} ({date_range})"

# d. Duplicate the sheet
    new_sheet = wks_SOG.duplicate_sheet(
        source_sheet_id=template_sheet.id,
        insert_sheet_index=(week_number + n_template_sheets),
        new_sheet_name=title_string
    )

    # e. Get Row Offsets
    Date_Column, Notes_Column, Title_Column, Leader_Column, Start_Time_Column, End_Time_Column, Description_Column, Location_Column, Points_Column, Category_Column, Event_ID_Column, Key_Column, Event_Start_Row, Event_End_Row, Fixed_Box_Start_Row, Fixed_Box_End_Row = Row_Offset(wks_SOG, week_number, n_template_sheets)

    # f. Prepare Date Values as Excel Serial Numbers
    date_values = []
    for i in range(7):
        day = monday_date + timedelta(days=i)
        date_values.append([to_excel_date(day)])

    # g. Construct Range (Handling 1-based indexing for gspread)
    # We use Date_Column + 1 because gspread/Google API is 1-based (A=1, B=2)
    col_letter = gspread.utils.rowcol_to_a1(1, Date_Column + 1).replace("1", "")
    cell_range = f"{col_letter}{Event_Start_Row+1}:{col_letter}{Event_End_Row+1}"

    # h. Update values and Apply proper Date Formatting
    # value_input_option='USER_ENTERED' allows Google to recognize the serial number
    new_sheet.update(cell_range, date_values, value_input_option='USER_ENTERED')
    
    # This applies the visual mask: "Monday, December 1"
    new_sheet.format(cell_range, {
        "numberFormat": {
            "type": "DATE",
            "pattern": "dddd, MMMM d"
        },
        "horizontalAlignment": "CENTER"
    })

    print(f"Successfully created: {title_string}")

1


C:\Users\Alex\AppData\Local\Temp\ipykernel_5104\2511812738.py:67: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  new_sheet.update(cell_range, date_values, value_input_option='USER_ENTERED')


Successfully created: Week 15 (December 8-14)


In [ ]:

week_number = 14+2
cal_data = pd.DataFrame(wks_SOG.get_worksheet(week_number).get_all_values(value_render_option='UNFORMATTED_VALUE'))[0:][:]

header_not_found = True
header_index = 0
while header_not_found:
    headers = cal_data.iloc[header_index].values
    if("Date" in headers):
        header_not_found = False
        break
    else:
        header_index+=1
        if(header_index > 100):
            break

headers = cal_data.iloc[header_index].values
cal_data.columns = headers
Date_Column = cal_data.columns.get_loc("Date")
Notes_Column = cal_data.columns.get_loc("Notes")
Title_Column = cal_data.columns.get_loc("Workshop Title")
Leader_Column = cal_data.columns.get_loc("Led By")
Start_Time_Column = cal_data.columns.get_loc("Start Time")
End_Time_Column = cal_data.columns.get_loc("End Time")
Description_Column = cal_data.columns.get_loc("Description")
Location_Column = cal_data.columns.get_loc("Location/Link")
Points_Column = cal_data.columns.get_loc("Points")
Category_Column = cal_data.columns.get_loc("Category")
Event_ID_Column = cal_data.columns.get_loc("Event ID")
Key_Column = cal_data.columns.get_loc("Key")

Date_arr = cal_data[headers[Date_Column]][header_index+1:][:]
end_not_found = True
end_index = 1
while end_not_found:
    if (not isinstance(Date_arr[end_index+header_index], int) and Date_arr[end_index+header_index]!=""):
        row_check = cal_data.iloc[end_index+header_index-1].values
        if all(item == "" for item in row_check):
            end_index = end_index-1
            end_not_found = False
            break
        else:
            end_index = end_index
            end_not_found = False
            break
    else:
        end_index+=1
        if(end_index > 100):
            break

# print(cal_data.iloc[end_index+header_index-1].values)
# print(cal_data.iloc[end_index+header_index+1].values)
# print(cal_data[headers[Date_Column]][header_index+1:][:])

Event_Start_Row = header_index+1
Event_End_Row = end_index+header_index-1
Fixed_Box_Start_Row = end_index+header_index+1
Fixed_Box_End_Row = len(cal_data[headers[Date_Column]])-1

14    s
Name: Notes, dtype: object
